In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import  warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv("car_prices.csv")

data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [4]:
X = data[["model_year","milage"]]
y = data["price"]

a = data["price"].mean()
a 

np.float64(39218.443332780575)

In [13]:
y.max()

np.int64(2954083)

In [14]:
y.min()

np.int64(2000)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=100)

model = LinearRegression()

kf = KFold(n_splits=5, shuffle=True, random_state=0)

rmse_scores = np.sqrt(-cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error'))


model.fit(X_train, y_train)

y_pred = model.predict(X_test)

RMSE = np.sqrt(mean_squared_error(y_test, y_pred))




print("RMSE Y_test: ", RMSE)

print("Mean RMSE for CV test: ", np.mean(rmse_scores))

RMSE Y_test:  68253.31178232699
Mean RMSE for CV test:  69228.11183073233


In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

ridge = Ridge()

ridge_params = {
    'alpha': [0.01, 1.0, 1000, 10000],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
}

ridge_grid = GridSearchCV(ridge, 
                          ridge_params, 
                          cv=5, 
                          scoring='neg_mean_squared_error')
ridge_grid.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.01, 1.0, 1000, 10000],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg', 'sag', 'saga', 'lbfgs']},
             scoring='neg_mean_squared_error')

In [56]:
print("Best parameters for Ridge:", ridge_grid.best_params_)
print("Best cross-validation RMSE for Ridge:", np.sqrt(-ridge_grid.best_score_))



ridge_best = ridge_grid.best_estimator_
y_pred_ridge = ridge_best.predict(X_test)
print("Test RMSE for Ridge:", mean_squared_error(y_test, y_pred_ridge, squared=False))


Best parameters for Ridge: {'alpha': 10000, 'solver': 'svd'}
Best cross-validation RMSE for Ridge: 70720.80772410984
Test RMSE for Ridge: 69352.12323233963


In [63]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)


elasticnet = ElasticNet()

elasticnet_params = {
    'alpha': np.linspace(0.1, 10, 100),
    'l1_ratio': np.linspace(0.1, 1.0, 100),
}

elasticnet_grid = GridSearchCV(elasticnet, 
                                     elasticnet_params, 
                                     cv=5, 
                                     scoring='neg_mean_squared_error', 
                                     n_jobs=8)
elasticnet_grid.fit(X_train, y_train)

print("Best parameters for ElasticNet:", elasticnet_grid.best_params_)
print("Best cross-validation RMSE for ElasticNet:", np.sqrt(-elasticnet_grid.best_score_))


elasticnet_best = elasticnet_grid.best_estimator_
y_pred_elasticnet = elasticnet_best.predict(X_test)
print("Test RMSE for ElasticNet:", mean_squared_error(y_test, y_pred_elasticnet, squared=False))


Best parameters for ElasticNet: {'alpha': np.float64(0.8), 'l1_ratio': np.float64(0.6545454545454545)}
Best cross-validation RMSE for ElasticNet: 69547.6643787254
Test RMSE for ElasticNet: 71846.42850633609


In [60]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from scipy.stats import uniform


elasticnet = ElasticNet()


param_dist = {
    'alpha': uniform(loc=0, scale=100),
    'l1_ratio': uniform(loc=0, scale=1)
}


random_search = RandomizedSearchCV(elasticnet, param_distributions=param_dist, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42)


random_search.fit(X_train, y_train)


best_params_random = random_search.best_params_
best_mse_random = -random_search.best_score_
best_rmse_random = np.sqrt(best_mse_random)

print("Best Parameters from RandomizedSearchCV:", best_params_random)
print("Best RMSE from RandomizedSearchCV:", best_rmse_random)


Best Parameters from RandomizedSearchCV: {'alpha': np.float64(0.5522117123602399), 'l1_ratio': np.float64(0.8154614284548342)}
Best RMSE from RandomizedSearchCV: 74431.81079727007


In [61]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from scipy.stats import norm


elasticnet = ElasticNet()

 
param_dist = {
    'alpha': norm(loc=0.3, scale=0.1),   
    'l1_ratio': norm(loc=0.573, scale=0.1)   
}


random_search = RandomizedSearchCV(elasticnet, param_distributions=param_dist, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=100)


random_search.fit(X_train, y_train)


best_params_random = random_search.best_params_
best_mse_random = -random_search.best_score_
best_rmse_random = np.sqrt(best_mse_random)

print("Best Parameters from RandomizedSearchCV:", best_params_random)
print("Best RMSE from RandomizedSearchCV:", best_rmse_random)


Best Parameters from RandomizedSearchCV: {'alpha': np.float64(0.29239765342753765), 'l1_ratio': np.float64(0.5733957593987599)}
Best RMSE from RandomizedSearchCV: 74431.81113345879


In [62]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from scipy.stats import norm


elasticnet = ElasticNet()


param_dist = {
    'alpha': norm(loc=0.552, scale=0.1),   
    'l1_ratio': norm(loc=0.815, scale=0.2)   
}


random_search = RandomizedSearchCV(elasticnet, param_distributions=param_dist, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=100)


random_search.fit(X_train, y_train)


best_params_random = random_search.best_params_
best_mse_random = -random_search.best_score_
best_rmse_random = np.sqrt(best_mse_random)

print("Best Parameters from RandomizedSearchCV:", best_params_random)
print("Best RMSE from RandomizedSearchCV:", best_rmse_random)


Best Parameters from RandomizedSearchCV: {'alpha': np.float64(0.5443976534275378), 'l1_ratio': np.float64(0.8157915187975198)}
Best RMSE from RandomizedSearchCV: 74431.81115974179
